# transformer istrain

Q: Does a Transformer know if it being trained? This has implications on AI safety.

Hypothesis: dropout "leaks" the train/eval phase bit.

Reference: https://twitter.com/karpathy/status/1635049541534879745

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# repro
torch.manual_seed(42);

In [ ]:

# create a toy transformer network doing BCE loss on last token
C = 64 # num channels

class TinyTransformer(nn.Module):
    def __init__(self, dropout):
        super(TinyTransformer, self).__init__()
        # random small encoder decoder transformer
        self.transformer = nn.Transformer(d_model=C, nhead=4, 
                       num_encoder_layers=4, num_decoder_layers=4,
                       dim_feedforward=C*4, dropout=dropout)
        self.fc = nn.Linear(C, 1)
    def forward(self, xe, xd):
        # forward the transformer
        x = self.transformer(xe, xd)
        # select the last time step to make the prediction
        x = x[:, -1, :]
        # forward the classifier
        x = self.fc(x)
        return x


In [ ]:
def train_model(model):

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # training loop
    B, T = 8, 4
    steps = 300

    for n in range(steps):

        # zero grad
        optimizer.zero_grad()

        # phase 1: train mode
        xe = torch.randn(B, T, C) # B,T,C for encoder
        xd = torch.randn(B, T, C) # B,T,C for decoder
        model.train()
        x = model(xe, xd)
        y = torch.ones(B, 1) # positive label: we are training
        loss = F.binary_cross_entropy_with_logits(x, y)
        loss.backward()
        if n % 100 == 0 or n == steps-1:
            print(f"{n} loss in phase 1: {loss.item()}")

        # phase 2: eval mode
        xe = torch.randn(B, T, C) # B,T,C for encoder
        xd = torch.randn(B, T, C) # B,T,C for decoder
        model.eval()
        x = model(xe, xd)
        y = torch.zeros(B, 1) # negative label: we are not training
        loss = F.binary_cross_entropy_with_logits(x, y)
        loss.backward()
        if n % 100 == 0 or n == steps-1:
            print(f"{n} loss in phase 2: {loss.item()}")

        # update
        optimizer.step()



In [ ]:
def eval_model(model):

    # evaluate accuracy on some synthetic test data
    corrects = []
    for test in range(200):
        
        # dummy input
        B, T = 1, 4
        xe = torch.randn(B, T, C) # B,T,C for encoder
        xd = torch.randn(B, T, C) # B,T,C for decoder

        # set network into train/eval phase
        phase = test % 2
        model.train() if phase == 1 else model.eval()
        
        # predict mode
        x = model(xe, xd)
        y = torch.sigmoid(x)
        pred = 1 if y.item() > 0.5 else 0
        
        # print(f"{test} gt: {phase}, pred: {pred}, correct: {phase == pred}")
        corrects.append(float(phase == pred))

    print(f"test accuracy {torch.tensor(corrects).mean().item()*100}%")


In [ ]:
# with dropout > 0.0 this should work, i.e. accuracy >> 50%
model = TinyTransformer(dropout=0.2)
train_model(model)
eval_model(model)

In [ ]:
# with dropout of 0 this should not work, i.e. accuracy ~= 50%
model = TinyTransformer(dropout=0.0)
train_model(model)
eval_model(model)